In [1]:
import pandas as pd
import numpy as np
import os

from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [2]:
source = 'source/UNPD_WCU2015_CP_Country Data Survey-Based.xlsx'

In [32]:
data = pd.read_excel(source, skiprows=3, na_values='..')

In [34]:
data = data.iloc[:, :19]

In [5]:
data.head()

,Country,ISO Code,Year(s),Age,Any method,Modern methods,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Traditional methods,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,NaN,NaN,NaN,NaN,NaN,Any modern method,Sterilization,NaN,Pill,Injectable,IUD,Male condom,Vaginal barrier methods,Implant,Other modern methods,Any traditional method,Rhythm,Withdrawal,Other traditional methods
1,NaN,NaN,NaN,NaN,NaN,NaN,Female,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,4.0,1972-1974,15-44,1.6,1.6,NaN,NaN,1,NaN,0.4,NaN,0.2,NaN,0,NaN,NaN,NaN,NaN
3,Afghanistan,4.0,2000,12-49,4.9,3.6,0.8,0.1,1.2,1.3,0.3,NaN,NaN,NaN,0,1.2,0.4,0.5,0.3
4,Afghanistan,4.0,2003,<50,10.3,8.7,NaN,NaN,5,2.2,NaN,0.6,NaN,NaN,0.9,1.6,NaN,NaN,NaN


In [6]:
# country

In [6]:
country = data[['Country', 'ISO Code']].copy()

In [7]:
country = country.drop_duplicates().dropna().copy()

In [8]:
country.columns = ['name', 'iso_code']

In [9]:
country.iso_code = country.iso_code.map(int)

In [10]:
country.to_csv('../ddf--entities--iso_code.csv', index=False)

In [16]:
# age

In [11]:
age = data[['Age']].drop_duplicates().copy()

In [12]:
age['age'] = age['Age'].map(to_concept_id)

In [13]:
age = age.dropna()

In [14]:
age.columns = ['name', 'age']

In [15]:
age.to_csv('../ddf--entities--age.csv', index=False)

In [ ]:
# methods

In [16]:
modern = data.iloc[0, 6:15].values

In [17]:
modern

array(['Sterilization', nan, 'Pill', 'Injectable', 'IUD', 'Male condom',
       'Vaginal barrier methods', 'Implant', 'Other modern methods'], dtype=object)

In [18]:
modern[0] = 'Female Sterilization'
modern[1] = 'Male Sterilization'

In [19]:
modern 

array(['Female Sterilization', 'Male Sterilization', 'Pill', 'Injectable',
       'IUD', 'Male condom', 'Vaginal barrier methods', 'Implant',
       'Other modern methods'], dtype=object)

In [20]:
trad = data.iloc[0, 16:].values

In [21]:
trad

array(['Rhythm', 'Withdrawal', 'Other traditional methods'], dtype=object)

In [22]:
methods = pd.DataFrame(np.r_[modern, trad], columns=['name'])

In [23]:
methods['method'] = methods['name'].map(to_concept_id)

In [24]:
methods = methods.set_index('name')

In [25]:
methods['is--modern_method'] = False
methods['is--traditional_method'] = False

In [26]:
methods.loc[modern, 'is--modern_method'] = True
methods.loc[trad, 'is--traditional_method'] = True

In [27]:
methods = methods.reset_index()

In [28]:
methods

,name,method,is--modern_method,is--traditional_method
0,Female Sterilization,female_sterilization,True,False
1,Male Sterilization,male_sterilization,True,False
2,Pill,pill,True,False
3,Injectable,injectable,True,False
4,IUD,iud,True,False
5,Male condom,male_condom,True,False
6,Vaginal barrier methods,vaginal_barrier_methods,True,False
7,Implant,implant,True,False
8,Other modern methods,other_modern_methods,True,False
9,Rhythm,rhythm,False,True


In [29]:
methods.to_csv('../ddf--entities--method.csv', index=False)

In [ ]:
# concepts

In [124]:
cdf = pd.DataFrame([['name', 'Name', 'string'],
                    ['iso_code', 'ISO Code', 'entity_domain'],
                    ['method', 'Methods', 'entity_domain'],
                    ['modern_method', 'Modern methods', 'entity_set'],
                    ['traditional_method', 'Traditional methods', 'entity_set'],
                    ['age', 'Age', 'entity_domain'],
                    ['year', 'Year(s)', 'time'],
                    ['domain', 'Domain', 'string'],
                    ['contraceptive_prevalence', 'Contraceptive Prevalence', 'measure']
                   ], columns=['concept', 'name', 'concept_type'])

In [125]:
cdf['domain'] = np.nan

In [126]:
cdf.loc[[3, 4], 'domain'] = 'method'

In [127]:
cdf

,concept,name,concept_type,domain
0,name,Name,string,NaN
1,iso_code,ISO Code,entity_domain,NaN
2,method,Methods,entity_domain,NaN
3,modern_method,Modern methods,entity_set,method
4,traditional_method,Traditional methods,entity_set,method
5,age,Age,entity_domain,NaN
6,year,Year(s),time,NaN
7,domain,Domain,string,NaN
8,contraceptive_prevalence,Contraceptive Prevalence,measure,NaN


In [128]:
cdf.to_csv('../ddf--concepts.csv', index=False)

In [75]:
# datapoints

In [35]:
data.columns

Index(['Country', 'ISO Code', 'Year(s)', 'Age', 'Any method', 'Modern methods',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Traditional methods', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'],
      dtype='object')

In [36]:
for i in range(len(data.columns)):
         
    if not data.ix[0].isnull().ix[i]:
        name1 = data.iloc[0, i]
        
    else:
        name1 = ''
        
    if not data.ix[1].isnull().ix[i]:
        name = data.iloc[1, i] + ' ' + name1

    else:
        name = name1
    
    if name:
        data = data.rename(columns={data.columns[i]: name})


In [37]:
data.columns

Index(['Country', 'ISO Code', 'Year(s)', 'Age', 'Any method',
       'Any modern method', 'Female Sterilization', 'Male ', 'Pill',
       'Injectable', 'IUD', 'Male condom', 'Vaginal barrier methods',
       'Implant', 'Other modern methods', 'Any traditional method', 'Rhythm',
       'Withdrawal', 'Other traditional methods'],
      dtype='object')

In [92]:
dps = data.drop([0, 1]).drop(['Any method', 'Any modern method', 'Any traditional method'], axis=1).copy()

In [94]:
dps = dps.drop(['Country'], axis=1)

In [96]:
dps.columns = list(map(to_concept_id, dps.columns))

In [98]:
dps = dps.rename(columns={'year_s': 'year'})

In [100]:
dps.iso_code = dps.iso_code.map(int)

In [101]:
dps = dps.set_index(['iso_code', 'year', 'age'])

In [105]:
dps = dps.stack().reset_index()

In [107]:
dps.columns = ['iso_code', 'year', 'age', 'method', 'contraceptive_prevalence']

In [109]:
dps.to_csv('../ddf--datapoints--contraceptive_prevalence--by--iso_code--age--year.csv', index=False)

In [129]:
create_index_file('../')

,key,value,file
0,concept,name,ddf--concepts.csv
1,concept,concept_type,ddf--concepts.csv
2,concept,domain,ddf--concepts.csv
0,"iso_code,age,year",contraceptive_prevalence,ddf--datapoints--contraceptive_prevalence--by-...
0,age,name,ddf--entities--age.csv
0,iso_code,name,ddf--entities--iso_code.csv
0,method,name,ddf--entities--method.csv
1,method,is--modern_method,ddf--entities--method.csv
2,method,is--traditional_method,ddf--entities--method.csv
